# OHW Github analysis

### A meta-hackweek hack

This is a rough and ready approach to measuring the impact of OHW on participants' Github activity

It goes without saying that **commits =/= work done on a project**. I freely admit that many of my commits are nonsense! This is just a fun side project for me to explore the github API and try some simple analysis methods.

First off, to access the [Github API](https://docs.github.com/) you'll need to edit the credentials file `credentials.json` to supply your username and a [Github access token](https://github.blog/2013-05-16-personal-api-tokens/).

Once you have supplied these creds, you are still limited to [5000 requests per hour](https://docs.github.com/en/rest/overview/resources-in-the-rest-api#rate-limiting) so, if you get bounced by the API, leave it some time to cool off. Without credentials the limit is far lower and you will would soon generate an error message like this

![](rate-limit-anon.png)

Obviously I have not provided my own credentials! I'm not sure what action github would take if a DDOS on their API originated from my account, but I'm not willing to find out.

### Let's get scraping!

In [1]:
import json
import requests
from collections import Counter
import pandas as pd
import numpy as np

In [2]:
credentials = json.loads(open('credentials.json').read()) #don't forget to add your creds here!

username = credentials['username']
token = credentials['token']

For a start, let's use the API to get some details on my account

In [3]:
user_data = requests.get('https://api.github.com/users/' + credentials['username'],auth = (username,token)).json()
user_data

{'login': 'callumrollo',
 'id': 28703282,
 'node_id': 'MDQ6VXNlcjI4NzAzMjgy',
 'avatar_url': 'https://avatars0.githubusercontent.com/u/28703282?v=4',
 'gravatar_id': '',
 'url': 'https://api.github.com/users/callumrollo',
 'html_url': 'https://github.com/callumrollo',
 'followers_url': 'https://api.github.com/users/callumrollo/followers',
 'following_url': 'https://api.github.com/users/callumrollo/following{/other_user}',
 'gists_url': 'https://api.github.com/users/callumrollo/gists{/gist_id}',
 'starred_url': 'https://api.github.com/users/callumrollo/starred{/owner}{/repo}',
 'subscriptions_url': 'https://api.github.com/users/callumrollo/subscriptions',
 'organizations_url': 'https://api.github.com/users/callumrollo/orgs',
 'repos_url': 'https://api.github.com/users/callumrollo/repos',
 'events_url': 'https://api.github.com/users/callumrollo/events{/privacy}',
 'received_events_url': 'https://api.github.com/users/callumrollo/received_events',
 'type': 'User',
 'site_admin': False,
 'n

Now, I'll try the account of one of my collaborators `ocefpaf`. You can specify any user, though the information returned is less than when you look at your own account

In [4]:
data = requests.get('https://api.github.com/users/' + 'ocefpaf',auth = (username,token)).json()
data

{'login': 'ocefpaf',
 'id': 950575,
 'node_id': 'MDQ6VXNlcjk1MDU3NQ==',
 'avatar_url': 'https://avatars1.githubusercontent.com/u/950575?v=4',
 'gravatar_id': '',
 'url': 'https://api.github.com/users/ocefpaf',
 'html_url': 'https://github.com/ocefpaf',
 'followers_url': 'https://api.github.com/users/ocefpaf/followers',
 'following_url': 'https://api.github.com/users/ocefpaf/following{/other_user}',
 'gists_url': 'https://api.github.com/users/ocefpaf/gists{/gist_id}',
 'starred_url': 'https://api.github.com/users/ocefpaf/starred{/owner}{/repo}',
 'subscriptions_url': 'https://api.github.com/users/ocefpaf/subscriptions',
 'organizations_url': 'https://api.github.com/users/ocefpaf/orgs',
 'repos_url': 'https://api.github.com/users/ocefpaf/repos',
 'events_url': 'https://api.github.com/users/ocefpaf/events{/privacy}',
 'received_events_url': 'https://api.github.com/users/ocefpaf/received_events',
 'type': 'User',
 'site_admin': False,
 'name': 'Filipe',
 'company': None,
 'blog': 'http://o

We can see a user's core stats. How about their commits and other actions taken? Simple append `/events` to the request query

In [5]:
data = requests.get('https://api.github.com/users/' + 'callumrollo' +'/events',auth = (username,token)).json()
data[0]

{'id': '13302795258',
 'type': 'PushEvent',
 'actor': {'id': 28703282,
  'login': 'callumrollo',
  'display_login': 'callumrollo',
  'gravatar_id': '',
  'url': 'https://api.github.com/users/callumrollo',
  'avatar_url': 'https://avatars.githubusercontent.com/u/28703282?'},
 'repo': {'id': 270385780,
  'name': 'callumrollo/callumrollo.github.io',
  'url': 'https://api.github.com/repos/callumrollo/callumrollo.github.io'},
 'payload': {'push_id': 5582281501,
  'size': 1,
  'distinct_size': 1,
  'ref': 'refs/heads/master',
  'head': 'ac52dfe8684d2b80e48d1398e2739874731400dd',
  'before': '448df7c51ec07f0113a40ea86b2d0a9866a5999c',
  'commits': [{'sha': 'ac52dfe8684d2b80e48d1398e2739874731400dd',
    'author': {'email': 'c.rollo@outlook.com', 'name': 'Callum Rollo'},
    'message': 'Generate Pelican site',
    'distinct': True,
    'url': 'https://api.github.com/repos/callumrollo/callumrollo.github.io/commits/ac52dfe8684d2b80e48d1398e2739874731400dd'}]},
 'public': True,
 'created_at': '20

Note that the Github API lists only the 30 most recent events

In [6]:
len(data)

30

To get at more events, we use a short loop to access subsequent pages of results. I found out the hard way that the API restricts you to 10 pages.

In [7]:
tgt_user = 'callumrollo'
base_url = 'https://api.github.com/users/' + tgt_user +'/events'
url = base_url
url_list = [base_url]
data = []
page_no = 1
repos_data = []
total_fetched = 0
while (True):
    response = requests.get(url,auth = (username,token)).json()
    data = data + response
    events_fetched = len(response)
    total_fetched = events_fetched + total_fetched
    print(f"Page: {page_no} total events fetched: {total_fetched}")
    
    if total_fetched == 300:
        print(f"\nAPI maxed out! https://docs.github.com/v3/#pagination\n\
        returning only most recent 300 events by {tgt_user}")
        print(f"\nevents span the range \n{data[-1]['created_at']}\n{data[0]['created_at']}")
        break
    
    if (events_fetched == 30):
        page_no = page_no + 1
        url = base_url + '?page=' + str(page_no)
        url_list.append(url)
    else:
        print(f"\n{tgt_user}: all your events are belong to us now")
        print(f"\nevents span the range \n{data[-1]['created_at']}\n{data[0]['created_at']}")
        break

Page: 1 total events fetched: 30
Page: 2 total events fetched: 60
Page: 3 total events fetched: 90
Page: 4 total events fetched: 120
Page: 5 total events fetched: 150
Page: 6 total events fetched: 180
Page: 7 total events fetched: 210
Page: 8 total events fetched: 240
Page: 9 total events fetched: 270
Page: 10 total events fetched: 299

callumrollo: all your events are belong to us now

events span the range 
2020-06-01T11:51:26Z
2020-08-25T14:42:16Z


This system logs all events: commits, issues, PRs, forks, stars etc. We are only interested in commits.

These are referred to as `PushEvent` in the json entry `type`

In [8]:
for event in data:
    print(event["type"])


PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
IssuesEvent
WatchEvent
WatchEvent
PushEvent
PushEvent
PushEvent
PullRequestEvent
PushEvent
PullRequestEvent
PushEvent
PullRequestEvent
PushEvent
PushEvent
PullRequestEvent
PullRequestEvent
PushEvent
PullRequestEvent
CreateEvent
PushEvent
PullRequestEvent
CreateEvent
PushEvent
IssuesEvent
IssuesEvent
PushEvent
PushEvent
PullRequestEvent
PullRequestEvent
CreateEvent
PushEvent
CreateEvent
PushEvent
IssueCommentEvent
IssueCommentEvent
IssuesEvent
IssueCommentEvent
PushEvent
PullRequestEvent
IssueCommentEvent
CreateEvent
IssuesEvent
IssueCommentEvent
IssueCommentEvent
PushEvent
PushEvent
PullRequestEvent
PushEvent
IssueCommentEvent
PushEvent
IssueCommentEvent
IssueCommentEvent
PushEvent
IssueCommentEvent
IssueCommentEvent
IssueCommentEvent
PullRequestEvent
PushEvent
ForkEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
IssueCommentEvent
ForkEvent
IssuesEvent
WatchEvent
PushEvent
PushEvent
PushEvent
PushEve

In [9]:
commit_events = []
for event in data:
    if event["type"] == "PushEvent":
        commit_events.append(event)
len(commit_events)

213

There are some complications. Not all of these commits in these events are by the Github user we are querying. For instance, some are commits by other users that our target user has merged in.

To work around this, we look through the payoad of each `PushEvent` and retain only the commits associated with the user we are interested in.

**n.b** this approach wil only work if the github username is the exact match for the name the author uses for their git commits. 

In [10]:
tgt_username = requests.get('https://api.github.com/users/' + credentials['username'],
                            auth = (username,token)).json()["name"]
tgt_username

'Callum Rollo'

In [11]:
user_commits = []
for event in commit_events:
    commit_list = event["payload"]["commits"]
    commit_list_author = []
    if len(commit_list)>0:
        for com_n in range(len(commit_list)):
            commit_username = commit_list[com_n]["author"]["name"]
            #print(commit_username)
            if commit_username == tgt_username:
                commit_list_author.append(commit_list[com_n])
        if commit_list_author:
            event["payload"]["commits"] = commit_list_author
            user_commits.append(event)

In [12]:
len(user_commits)

210

We can print out the name associated with the commits we have selected to confirm

In [13]:
number_commits = 0
for event in user_commits:
    commit_list = event["payload"]["commits"]
    for com_n in range(len(commit_list)):
            commit_username = commit_list[com_n]["author"]["name"]
            number_commits += 1
            print(commit_username)
    

Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo
Callum Rollo

In [14]:
print(f"From {len(data)} events we have extracted {number_commits} commits by {tgt_username}")

From 299 events we have extracted 241 commits by Callum Rollo


The final step of this (almost certainly imperfect) data cleaning is to get info on all the commits by this user. We will pull the author, message, SHA, url, repo and date into a pandas dataframe. 

In [15]:
df = pd.DataFrame()
for event in user_commits:
    for commit in event["payload"]["commits"]:
        commit_subset = {"id": event["id"],
                     "datetime" : event["created_at"],
                     "sha" : commit["sha"],
                     "message" : commit["message"],
                     "author" : commit["author"]["name"],
                     "url": commit["url"],
                     "repo": event["repo"]["name"]}
        df = df.append(commit_subset, ignore_index=True)

We index by datetime and have a look at our dataframe

In [16]:
df.index = pd.DatetimeIndex(df.datetime)
df = df.drop("datetime", axis=1)
df

,author,id,message,repo,sha,url
datetime,,,,,,
2020-08-25 14:42:16+00:00,Callum Rollo,13302795258,Generate Pelican site,callumrollo/callumrollo.github.io,ac52dfe8684d2b80e48d1398e2739874731400dd,https://api.github.com/repos/callumrollo/callu...
2020-08-25 14:41:45+00:00,Callum Rollo,13302788018,typesetting,callumrollo/callumrollo.github.io,d93f3bf0865ed6b562e2a3b71b0ec7cfd1e47c64,https://api.github.com/repos/callumrollo/callu...
2020-08-25 14:39:26+00:00,Callum Rollo,13302755051,Generate Pelican site,callumrollo/callumrollo.github.io,448df7c51ec07f0113a40ea86b2d0a9866a5999c,https://api.github.com/repos/callumrollo/callu...
2020-08-25 14:39:04+00:00,Callum Rollo,13302749789,hackweek article,callumrollo/callumrollo.github.io,8983e2c9fe8e3c5d2e29669ec81a9fe128f4417e,https://api.github.com/repos/callumrollo/callu...
2020-08-24 09:48:51+00:00,Callum Rollo,13286472652,added articles from OHW,callumrollo/callumrollo.github.io,79a10b6a21688d53f71e2dd1802499eb50343124,https://api.github.com/repos/callumrollo/callu...
...,...,...,...,...,...,...
2020-06-05 13:14:39+00:00,Callum Rollo,12541935216,Deployed dc9dbdd with MkDocs version: 1.1.2,ueapy/pythoncourse2020,9d8d94b00a71561d978d2524281e97a3dbf164c3,https://api.github.com/repos/ueapy/pythoncours...
2020-06-04 19:48:20+00:00,Callum Rollo,12534632581,Deployed 3236f45 with MkDocs version: 1.1.2,ueapy/pythoncourse2019,8228bc46ff11ab6a38d11248c76b6841b6b2dbc3,https://api.github.com/repos/ueapy/pythoncours...
2020-06-04 19:46:38+00:00,Callum Rollo,12534618123,Deployed 3236f45 with MkDocs version: 1.1.2,ueapy/pythoncourse2019,a49dcbffef4a8540002a9010b5920ae44c80274b,https://api.github.com/repos/ueapy/pythoncours...


Now we remove any repeated commits that may have snuck in by a deduplicating on the SHA checksum

**side note** the SHA chescksum uniquely identifies each commit. Even if you had commits by the same author to the same repo with the same message ("added stuff" or something similarly helpful) the SHA will differentiate the two. See more [here](https://www.lifewire.com/what-is-sha-1-2626011)

In [17]:
df = df.drop_duplicates(subset=['sha'])
df

,author,id,message,repo,sha,url
datetime,,,,,,
2020-08-25 14:42:16+00:00,Callum Rollo,13302795258,Generate Pelican site,callumrollo/callumrollo.github.io,ac52dfe8684d2b80e48d1398e2739874731400dd,https://api.github.com/repos/callumrollo/callu...
2020-08-25 14:41:45+00:00,Callum Rollo,13302788018,typesetting,callumrollo/callumrollo.github.io,d93f3bf0865ed6b562e2a3b71b0ec7cfd1e47c64,https://api.github.com/repos/callumrollo/callu...
2020-08-25 14:39:26+00:00,Callum Rollo,13302755051,Generate Pelican site,callumrollo/callumrollo.github.io,448df7c51ec07f0113a40ea86b2d0a9866a5999c,https://api.github.com/repos/callumrollo/callu...
2020-08-25 14:39:04+00:00,Callum Rollo,13302749789,hackweek article,callumrollo/callumrollo.github.io,8983e2c9fe8e3c5d2e29669ec81a9fe128f4417e,https://api.github.com/repos/callumrollo/callu...
2020-08-24 09:48:51+00:00,Callum Rollo,13286472652,added articles from OHW,callumrollo/callumrollo.github.io,79a10b6a21688d53f71e2dd1802499eb50343124,https://api.github.com/repos/callumrollo/callu...
...,...,...,...,...,...,...
2020-06-05 13:14:39+00:00,Callum Rollo,12541935216,Deployed dc9dbdd with MkDocs version: 1.1.2,ueapy/pythoncourse2020,9d8d94b00a71561d978d2524281e97a3dbf164c3,https://api.github.com/repos/ueapy/pythoncours...
2020-06-04 19:48:20+00:00,Callum Rollo,12534632581,Deployed 3236f45 with MkDocs version: 1.1.2,ueapy/pythoncourse2019,8228bc46ff11ab6a38d11248c76b6841b6b2dbc3,https://api.github.com/repos/ueapy/pythoncours...
2020-06-04 19:46:38+00:00,Callum Rollo,12534618123,Deployed 3236f45 with MkDocs version: 1.1.2,ueapy/pythoncourse2019,a49dcbffef4a8540002a9010b5920ae44c80274b,https://api.github.com/repos/ueapy/pythoncours...


### Scaling it up: work from a list of Github usernames

In [18]:
def gh_scrape(tgt_users, cred_file = 'credentials.json', verbose=True):
    """Simple scraping function
    Supply a list of github usersnames ['jane-doe', 'torvalds', 'satoshi_nakamoto']
    Returns a dataframe of unique commits by this user over the last 90 days
    Assumes that the github user is the user with most git commits associated with their github profile
    Rate limited by the Github API to 300 events
    Requires you to supply a Github API token in an credentials.json file
    verbose switch prints a line for each user with the number of events and commits found
    Returns a pandas dataframe of commit info for all usernames in supplied list
    """
    credentials = json.loads(open('credentials.json').read())
    username = credentials['username']
    token = credentials['token']
    
    df = pd.DataFrame()
    
    for tgt_user in tgt_users:
        base_url = 'https://api.github.com/users/' + tgt_user +'/events'
        url = base_url
        url_list = [base_url]
        data = []
        page_no = 1
        repos_data = []
        total_fetched = 0
        while (True):
            response = requests.get(url,auth = (username,token)).json()
            data = data + response
            events_fetched = len(response)
            total_fetched = events_fetched + total_fetched
            if total_fetched == 300:
                break
            if (events_fetched == 30):
                page_no = page_no + 1
                url = base_url + '?page=' + str(page_no)
                url_list.append(url)
            else:
                #print(f"\n{tgt_user}: fetched {total_fetched} events")
                #print(f"events span the range \n{data[0]['created_at']}\n{data[-1]['created_at']}")
                break
        commits_events = []
        for event in data:
            if event["type"] == "PushEvent":
                commits_events.append(event)
        # Working on the assumption that the most common commit username associated with this user is the one we want
        if len(commits_events)==0:
            continue
        commit_usernames_list = []
        for event in commits_events:
            commit_list = event["payload"]["commits"]
            if len(commit_list)>0:    
                for com_n in range(len(commit_list)):
                        commit_username = commit_list[com_n]["author"]["name"]       
                        commit_usernames_list.append(commit_username)

        c = Counter(commit_usernames_list)
        #print(c.most_common(3))
        most_common_username = c.most_common(1)[0][0]
        user_commits = []

        for event in commits_events:
            commit_list = event["payload"]["commits"]
            commit_list_author = []
            if len(commit_list)>0:    
                for com_n in range(len(commit_list)):
                    commit_username = commit_list[com_n]["author"]["name"]
                    if commit_username == most_common_username:
                        commit_list_author.append(commit_list[com_n])
                if commit_list_author:
                    event["payload"]["commits"] = commit_list_author
                    user_commits.append(event)

        for commit in user_commits:
            for com_n in range(len(commit["payload"]["commits"])):
                commit_detail = commit["payload"]["commits"][com_n]
                commit_subset = {"id": commit["id"],
                             "datetime" : commit["created_at"],
                             "sha" : commit_detail["sha"],
                             "message" : commit_detail["message"],
                             "author" : commit_detail["author"]["name"],
                             "url": commit_detail["url"],
                             "repo": commit["repo"]["name"]}
                df = df.append(commit_subset, ignore_index=True)
        df.index = pd.DatetimeIndex(df.datetime)
        #df = df.drop("datetime", axis=1)
        df = df.drop_duplicates(subset=['sha'])
        df
        if verbose:
            print(f"{tgt_user}: found {len(data)} events containing {len(user_commits)} unique commits by {most_common_username}\n")
    return df


In [19]:
users = ["callumrollo"]
df = gh_scrape(users)
df

callumrollo: found 299 events containing 210 unique commits by Callum Rollo



,author,datetime,id,message,repo,sha,url
datetime,,,,,,,
2020-08-25 14:42:16+00:00,Callum Rollo,2020-08-25T14:42:16Z,13302795258,Generate Pelican site,callumrollo/callumrollo.github.io,ac52dfe8684d2b80e48d1398e2739874731400dd,https://api.github.com/repos/callumrollo/callu...
2020-08-25 14:41:45+00:00,Callum Rollo,2020-08-25T14:41:45Z,13302788018,typesetting,callumrollo/callumrollo.github.io,d93f3bf0865ed6b562e2a3b71b0ec7cfd1e47c64,https://api.github.com/repos/callumrollo/callu...
2020-08-25 14:39:26+00:00,Callum Rollo,2020-08-25T14:39:26Z,13302755051,Generate Pelican site,callumrollo/callumrollo.github.io,448df7c51ec07f0113a40ea86b2d0a9866a5999c,https://api.github.com/repos/callumrollo/callu...
2020-08-25 14:39:04+00:00,Callum Rollo,2020-08-25T14:39:04Z,13302749789,hackweek article,callumrollo/callumrollo.github.io,8983e2c9fe8e3c5d2e29669ec81a9fe128f4417e,https://api.github.com/repos/callumrollo/callu...
2020-08-24 09:48:51+00:00,Callum Rollo,2020-08-24T09:48:51Z,13286472652,added articles from OHW,callumrollo/callumrollo.github.io,79a10b6a21688d53f71e2dd1802499eb50343124,https://api.github.com/repos/callumrollo/callu...
...,...,...,...,...,...,...,...
2020-06-05 13:14:39+00:00,Callum Rollo,2020-06-05T13:14:39Z,12541935216,Deployed dc9dbdd with MkDocs version: 1.1.2,ueapy/pythoncourse2020,9d8d94b00a71561d978d2524281e97a3dbf164c3,https://api.github.com/repos/ueapy/pythoncours...
2020-06-04 19:48:20+00:00,Callum Rollo,2020-06-04T19:48:20Z,12534632581,Deployed 3236f45 with MkDocs version: 1.1.2,ueapy/pythoncourse2019,8228bc46ff11ab6a38d11248c76b6841b6b2dbc3,https://api.github.com/repos/ueapy/pythoncours...
2020-06-04 19:46:38+00:00,Callum Rollo,2020-06-04T19:46:38Z,12534618123,Deployed 3236f45 with MkDocs version: 1.1.2,ueapy/pythoncourse2019,a49dcbffef4a8540002a9010b5920ae44c80274b,https://api.github.com/repos/ueapy/pythoncours...


Quite a few commits. What if we want solely the hackweek ones?

In [20]:
df['ohw20_repo'] = df['repo'].str.contains("ohw20")
sum(df['ohw20_repo'] )

19

### OHW analysis

Using the above function and a list of hackweek participants (not included) I grab github commits from the last 90 days

In [21]:
import csv
from itertools import chain
with open('ohw_participants.csv', newline='') as f:
    nest_list = list(csv.reader(f))
ohw_participants_list = list(chain.from_iterable(nest_list))
df_all = gh_scrape(ohw_participants_list, verbose=False)

print(f"total commits: {len(df_all)}")

total commits: 1536


That's a lot of commits! 

As you saw when I grabbed the data just from my username, it contains a lot of identifying information. Time for some anonymising

In [22]:
df_min = df_all[['author', 'message']].copy()
df_min['ohw20_repo'] = df_all['repo'].str.contains("ohw20")

In [23]:
uniq_names = np.unique(df_min.author.values)

for n, uniq_name in enumerate(uniq_names):
    df_min = df_min.replace({uniq_name: 'participant-'+str(n)})
    

In [24]:
df_min

,author,message,ohw20_repo
datetime,,,
2020-08-20 15:08:09+00:00,participant-20,Merge pull request #1 from cbirdferrer/patch-1...,False
2020-08-17 16:24:17+00:00,participant-20,deleting old presentation_figs file,True
2020-08-14 20:00:38+00:00,participant-20,adding improvements to interpolate notebook to...,True
2020-08-14 20:00:38+00:00,participant-20,merging changes,True
2020-08-14 20:00:38+00:00,participant-20,merging interpolate notebook,True
...,...,...,...
2020-08-07 15:58:21+00:00,participant-32,change read.md,False
2020-08-06 22:26:24+00:00,participant-32,change new2.md and create new.md,False
2020-08-06 22:25:02+00:00,participant-32,changes in new.md and new2.md,False


In [25]:
df_min.to_csv('ohw_anonymised.csv')

### What to do with this data? 
What cool analysis could we do? Grab the dataset `ohw_anonymised.csv` and give it a go.

I've started some analysis in the companion notebook `ohw_analysis.ipynb` in this directory

-----------------------------

# Going deeper

We can get all the details of a commit by delving deeper into the json structure accessed through the commit url.

**example:**

In [28]:
url = event['payload']['commits'][0]['url']
commit_detail = requests.get(url,auth = (username,token)).json()

The most interesting section is `files`. This gives a summary of the lines changed on each file altered in this commit

In [29]:
commit_detail['files']

[{'sha': 'f84a0380775fc5b627d7689da03a71a7f4c8f3a5',
  'filename': 'README.md',
  'status': 'modified',
  'additions': 1,
  'deletions': 1,
  'changes': 2,
  'blob_url': 'https://github.com/ueapy/pythoncourse2020-materials/blob/78bfe4806b8ce6166e7643b344e40e324d2c2412/README.md',
  'raw_url': 'https://github.com/ueapy/pythoncourse2020-materials/raw/78bfe4806b8ce6166e7643b344e40e324d2c2412/README.md',
  'contents_url': 'https://api.github.com/repos/ueapy/pythoncourse2020-materials/contents/README.md?ref=78bfe4806b8ce6166e7643b344e40e324d2c2412',
  'patch': '@@ -1,6 +1,6 @@\n Content of Ares and Cefas Python Course 2020 [website](<https://ueapy.github.io/pythoncourse2019>)\n \n-[![Python 3.7](https://img.shields.io/badge/python-3.7-blue.svg)]()\n+[![Python 3.8](https://img.shields.io/badge/python-3.8-blue.svg)]()\n [![Binder](http://mybinder.org/badge.svg)](http://mybinder.org:/repo/ueapy/pythoncourse2019-materials)\n \n ## License'}]

### Ideas for further analysis
- Use of different filetypes. Particularly .py vs .ipynb
- word cloud of commit messages
- check out non-commit activity: merge, PR, issue...
- geographical/timezone patterns
- how much "crunch" did we get before presentations on Friday?
- examine links between authors (who merged who? Comments mentioning issues?)

### Github is a rich mine of information
I hope this notebook serves as a good reminder that everything we put on it is public and scrapable, so write helpful commit messages!